<a href="https://colab.research.google.com/github/kimdain0222/kimdain0222/blob/main/mimic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, f1_score
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')
import kagglehub
path = kagglehub.dataset_download("asjad99/mimiciii")

100%|██████████| 10.6M/10.6M [00:00<00:00, 105MB/s]

Extracting files...


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asjad99/mimiciii")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/mimiciii


In [5]:
df_PATIENTS = pd.read_csv('/kaggle/input/mimiciii/mimic-iii-clinical-database-demo-1.4/PATIENTS.csv')
df_CHARTEVENTS = pd.read_csv('/kaggle/input/mimiciii/mimic-iii-clinical-database-demo-1.4/CHARTEVENTS.csv')
df_ADMISSIONS = pd.read_csv('/kaggle/input/mimiciii/mimic-iii-clinical-database-demo-1.4/ADMISSIONS.csv')

In [6]:
df_PATIENTS[df_PATIENTS['subject_id']==10006]

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1


In [7]:
df_CHARTEVENTS[df_CHARTEVENTS['subject_id']==10006][:10]

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
101426,85780442,10006,142345,206504.0,69,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101427,85780443,10006,142345,206504.0,762,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101428,85780444,10006,142345,206504.0,916,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,Lidocaine,NaN,NaN,NaN,NaN,NaN,NaN
101429,85780445,10006,142345,206504.0,917,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,FEVER,NaN,NaN,NaN,NaN,NaN,NaN
101430,85780446,10006,142345,206504.0,919,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,MED,NaN,NaN,NaN,NaN,NaN,NaN
101431,85780447,10006,142345,206504.0,920,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101432,85780448,10006,142345,206504.0,924,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101433,85780449,10006,142345,206504.0,925,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,A,NaN,NaN,NaN,NaN,NaN,NaN
101434,85780450,10006,142345,206504.0,926,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,RCA,NaN,NaN,NaN,NaN,NaN,NaN
101435,85780451,10006,142345,206504.0,927,2164-10-23 21:10:00,2164-10-23 21:10:00,14077,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_ADMISSIONS[df_ADMISSIONS['subject_id']==10006]

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1


In [9]:
len(set(df_PATIENTS['subject_id']))

100

In [10]:
len(set(df_CHARTEVENTS['subject_id']))

99

In [11]:
len(set(df_ADMISSIONS['subject_id']))

100

In [12]:
set(df_ADMISSIONS['subject_id'])

{10006,
 10011,
 10013,
 10017,
 10019,
 10026,
 10027,
 10029,
 10032,
 10033,
 10035,
 10036,
 10038,
 10040,
 10042,
 10043,
 10044,
 10045,
 10046,
 10056,
 10059,
 10061,
 10064,
 10065,
 10067,
 10069,
 10074,
 10076,
 10083,
 10088,
 10089,
 10090,
 10093,
 10094,
 10098,
 10101,
 10102,
 10104,
 10106,
 10111,
 10112,
 10114,
 10117,
 10119,
 10120,
 10124,
 10126,
 10127,
 10130,
 10132,
 40124,
 40177,
 40204,
 40277,
 40286,
 40304,
 40310,
 40456,
 40503,
 40595,
 40601,
 40612,
 40655,
 40687,
 41795,
 41914,
 41976,
 41983,
 42033,
 42066,
 42075,
 42135,
 42199,
 42231,
 42275,
 42281,
 42292,
 42302,
 42321,
 42346,
 42367,
 42412,
 42430,
 42458,
 43735,
 43746,
 43748,
 43779,
 43798,
 43827,
 43870,
 43879,
 43881,
 43909,
 43927,
 44083,
 44154,
 44212,
 44222,
 44228}

In [19]:
def analyze_columns(df, df_name):
    """데이터프레임의 칼럼 분석 함수"""
    print(f"\n{'='*50}")
    print(f"📊 {df_name} 테이블 분석 ({len(df.columns)}개 칼럼, {len(df)}개 행)")
    print('='*50)

    # 1. 기본 정보 요약
    print("\n[1] 기본 정보:")
    display(df.head(2))

    # 2. 칼럼별 상세 분석
    print("\n[2] 칼럼 상세 분석:")
    column_info = pd.DataFrame({
        '결측값 수': df.isnull().sum(),
        '결측값 비율': df.isnull().mean().round(4),
        '고유값 수': df.nunique(),
        '데이터 타입': df.dtypes
    })
    display(column_info)

    # 3. 범주형 칼럼 값 분포
    print("\n[3] 범주형 칼럼 값 분포:")
    for col in df.select_dtypes(include='object'):
        if df[col].nunique() < 50:  # 고유값이 50개 미만인 경우만 출력
            print(f"\n🔹 {col} 분포:")
            display(df[col].value_counts(normalize=True).head(10))

    # 4. 수치형 칼럼 통계
    print("\n[4] 수치형 칼럼 통계:")
    if df.select_dtypes(include='number').shape[1] > 0:
        display(df.describe().T)
    else:
        print("수치형 칼럼 없음")

# 각 테이블 분석 실행
analyze_columns(df_PATIENTS, "PATIENTS")
analyze_columns(df_ADMISSIONS, "ADMISSIONS")
analyze_columns(df_CHARTEVENTS, "CHARTEVENTS")

# 추가: CHARTEVENTS의 주요 ITEMID 분석
print("\n\n🔬 CHARTEVENTS 항목 분석 (상위 20개 ITEMID)")
display(df_CHARTEVENTS['itemid'].value_counts().head(20))


📊 PATIENTS 테이블 분석 (8개 칼럼, 100개 행)

[1] 기본 정보:


,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1



[2] 칼럼 상세 분석:


,결측값 수,결측값 비율,고유값 수,데이터 타입
row_id,0,0.00,100,int64
subject_id,0,0.00,100,int64
gender,0,0.00,2,object
dob,0,0.00,99,object
dod,0,0.00,100,object
dod_hosp,30,0.30,70,object
dod_ssn,23,0.23,77,object
expire_flag,0,0.00,1,int64



[3] 범주형 칼럼 값 분포:

🔹 gender 분포:


...

,proportion
stopped,
NotStopd,0.99668
D/C'd,0.00332



[4] 수치형 칼럼 통계:


,count,mean,std,min,25%,50%,75%,max
row_id,758355.0,1.632063e+08,1.010304e+08,5279021.0,86019147.5,203564544.0,272516662.5,274493921.0
subject_id,758355.0,2.630801e+04,1.611066e+04,10006.0,10111.0,40124.0,42135.0,44228.0
hadm_id,758355.0,1.538517e+05,2.587283e+04,100375.0,138132.0,160445.0,172082.0,199395.0
icustay_id,758274.0,2.492040e+05,2.551028e+04,201006.0,232110.0,249805.0,271544.0,298685.0
itemid,758355.0,1.139230e+05,1.110885e+05,1.0,618.0,220045.0,224080.0,228414.0
cgid,758355.0,1.796099e+04,2.178703e+03,14023.0,15963.0,17873.0,19907.0,21570.0
valuenum,323884.0,8.157890e+01,3.911579e+03,-500.0,4.0,35.0,92.0,2222221.7
warning,382279.0,1.932097e-02,1.376507e-01,0.0,0.0,0.0,0.0,1.0
error,382279.0,1.695097e-03,4.113670e-02,0.0,0.0,0.0,0.0,1.0




🔬 CHARTEVENTS 항목 분석 (상위 20개 ITEMID)


,count
itemid,
220045,8094
220210,8056
220277,8053
220048,7414
211,7396
742,7349
646,7262
212,7095
618,7030
